# Initialize Truveta SDK

In [1]:
library(readr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(magrittr, warn.conflicts = FALSE)
library(stringr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(rlang, warn.conflicts = FALSE)
library(data.table, warn.conflicts = FALSE)
library(lubridate, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(truveta.notebook.study)

In [2]:
# Use only one statement below and comment out whichever you are not using.
# con <- create_connection(output_mode = "sparklyr")
con <- create_connection(output_mode = "sparkr")

In [3]:
study <- get_study(con)
# Use only one statement below and comment out whichever you are not using.
# population <- get_population(con, study, title='MCI')
population <- get_population(con, id='p-an3klyy7kz2u7hq3fngqv7v63i')

#population

In [4]:
# Get latest completed active snapshot.
snapshot <- get_latest_snapshot(con, population)
snapshot

In [ ]:
# Show tables in the snapshot.
get_tables(con, snapshot)

In [26]:
sql <- " SELECT a.DispenseQuantityUOMConceptId ,c.ConceptName,  count(distinct a.Id) as cnt
FROM MedicationDispense a join Concept c on (a.DispenseQuantityUOMConceptId = c.ConceptId)
GROUP BY DispenseQuantityUOMConceptId,ConceptName"

a <- load_sql_table(con,snapshot,query = sql)
display_df(a)

In [27]:
sql <- " SELECT a.StatusConceptId ,c.ConceptName,  count(distinct a.Id) as cnt
FROM MedicationDispense a join Concept c on (a.StatusConceptId = c.ConceptId)
GROUP BY StatusConceptId,ConceptName"

b <- load_sql_table(con,snapshot,query = sql)
display_df(b)

In [7]:
## get medication name first

donepezil = codeset_from_prose(con,snapshot, url = "/definitions/approved-ad-drug", variable_name = "donepezil")

create_view(donepezil, "med_codes")

In [ ]:
sql1 <- "

--EX: De-DUP based on essential variables and select subjects with exposure to DOI

--Only medications needed for analysis

--Filter concept maps and names for the medications wanted
WITH map as 
( SELECT mcm.Id, cc.*
FROM  MedicationCodeConceptMap mcm 
JOIN Concept cc on (mcm.CodeConceptId = cc.ConceptId)
WHERE mcm.CodeConceptId IN
(SELECT ConceptId
FROM med_codes
)
),


--Remove error, canceled,declined or stopped dispense 
med_base as 
(SELECT *
FROM MedicationDispense
WHERE StatusConceptId NOT IN (2989063, 2989065,2989060,2989064)

)


-- Join Dispense data with concepts
tb1 as
(
    SELECT DISTINCT d.PersonId, d.DispenseDateTime, d.CodeConceptId,
     d.DaysSupply, d.DispenseQuantity, d.DispenseQuantityUOMConceptId,
     mcm.ConceptName, mcm.Definition
     FROM med_base d
    INNER JOIN map mcm
      ON mcm.Id = d.CodeConceptMapId
),

-- To derive earliest and latest dispense date for each subject for each drug
tb3 as
(
    SELECT PersonId, min(DispenseDateTime) as DSSTDT, max(DispenseDateTime) as DSENDT, count(*) as TotNumDisRecDrug
    FROM tb1
    GROUP BY PersonId
),

-- EX: Merge the table 2 data with table 3 to get the DSSTDT, DSENDT
EX as
(
    SELECT d.PersonId, d.DispenseDateTime, d.CodeConceptId, d.ConceptName, d.DaysSupply, d.DispenseQuantity, d.DispenseQuantityUOMConceptId,
    t3.DSSTDT, t3.DSENDT
    FROM tb1 d LEFT JOIN tb3 as t3
     ON d.PersonId = t3. PersonId
)

-- EXQC: subject level data
tb4 as
(
   SELECT PersonId, min(DispenseDateTime) as DispenseDataFirstDT, max(DispenseDateTime) as DispenseDataLastDT, count(*) as TotNumDisRec
   FROM med_base
   GROUP BY PersonId
)

EXQC as
(
   SELECT t4.*, (DispenseDataFirstDT -  DispenseDataLastDT) as DISSDAYS, t3.DSSTDT, t3.DSENDT, 
   t3.DSSTDT - t3.DSENDT as DISSDAYSDRUG, t3.TotNumDisRecDrug, %s as TRT
   FROM tb4 t4 INNER JOIN tb3 t3
    ON t4.PersonId=t3.PersonId
)

"

In [ ]:
a <- load_sql_table(con,snapshot, query = sql)

In [32]:
sql2 <- "

--EX: De-DUP based on essential variables and select subjects with exposure to DOI

--Only medications needed for analysis

--Filter concept maps and names for the medications wanted
WITH map as 
( SELECT mcm.Id, cc.*
FROM  MedicationCodeConceptMap mcm 
JOIN Concept cc on (mcm.CodeConceptId = cc.ConceptId)
WHERE mcm.CodeConceptId IN
(SELECT ConceptId
FROM med_codes
)
),


--Remove error, canceled,declined or stopped dispense 
med_base as 
(SELECT *
FROM MedicationDispense
WHERE StatusConceptId NOT IN (2989063, 2989065,2989060,2989064)

),


-- Join Dispense data with concepts
tb1 as
(
    SELECT DISTINCT d.PersonId, d.DispenseDateTime, d.Id as DispenseId,
     d.DaysSupply, d.DispenseQuantity, d.DispenseQuantityUOMConceptId,
     mcm.ConceptName as MedicationName, mcm.ConceptDefinition as MedicationDefinition
     FROM med_base d
    INNER JOIN map mcm
      ON mcm.Id = d.CodeConceptMapId
),


---tb2 The overall dispense data needed for final EX data
tb2 as (
    SELECT  d.*, c.ConceptName as DispenseQuantityUOM 
    FROM tb1 d join Concept c on (d.DispenseQuantityUOMConceptId = c.ConceptId)
),
-- To derive earliest and latest dispense date for each subject for each drug
tb3 as
(
    SELECT PersonId, min(DispenseDateTime) as DSSTDT, max(DispenseDateTime) as DSENDT,
     count(distinct DispenseId) as TotNumDisRecDrug
    FROM tb2
    GROUP BY PersonId
),

tb4 as (
   SELECT a.*, DATEDIFF( b.DSENDT,b.DSSTDT) as DISSDAYSDRUG,
    b.TotNumDisRecDrug, 'Donepezil' as TRT
   FROM tb2 a join tb3 b on (a.PersonId = b.PersonId)
)

SELECT *
FROM tb4
"

In [33]:
med1 <- load_sql_table(con,snapshot, query = sql2)

display_df(med1) # this is the EX data

## a proposed use of this sql function

In [ ]:
get_exposure <- function(codes , name = "drugA"){

    create_view(codes, "med_codes")
    sql2 <- "

--EX: De-DUP based on essential variables and select subjects with exposure to DOI

--Only medications needed for analysis

--Filter concept maps and names for the medications wanted
WITH map as 
( SELECT mcm.Id, cc.*
FROM  MedicationCodeConceptMap mcm 
JOIN Concept cc on (mcm.CodeConceptId = cc.ConceptId)
WHERE mcm.CodeConceptId IN
(SELECT ConceptId
FROM med_codes
)
),


--Remove error, canceled,declined or stopped dispense 
med_base as 
(SELECT *
FROM MedicationDispense
WHERE StatusConceptId NOT IN (2989063, 2989065,2989060,2989064)

),


-- Join Dispense data with concepts
tb1 as
(
    SELECT DISTINCT d.PersonId, d.DispenseDateTime, d.Id as DispenseId,
     d.DaysSupply, d.DispenseQuantity, d.DispenseQuantityUOMConceptId,
     mcm.ConceptName as MedicationName, mcm.ConceptDefinition as MedicationDefinition
     FROM med_base d
    INNER JOIN map mcm
      ON mcm.Id = d.CodeConceptMapId
),


---tb2 The overall dispense data needed for final EX data
tb2 as (
    SELECT  d.*, c.ConceptName as DispenseQuantityUOM 
    FROM tb1 d join Concept c on (d.DispenseQuantityUOMConceptId = c.ConceptId)
),
-- To derive earliest and latest dispense date for each subject for each drug
tb3 as
(
    SELECT PersonId, min(DispenseDateTime) as DSSTDT, max(DispenseDateTime) as DSENDT,
     count(distinct DispenseId) as TotNumDisRecDrug
    FROM tb2
    GROUP BY PersonId
),

tb4 as (
   SELECT a.*, DATEDIFF( b.DSENDT,b.DSSTDT) as DISSDAYSDRUG,
    b.TotNumDisRecDrug, '%s' as TRT
   FROM tb2 a join tb3 b on (a.PersonId = b.PersonId)
)

SELECT *
FROM tb4
"
sql <- sprintf(sql2, name)

re <- load_sql_table(con,snapshot, query = sql)

print(paste( "extracting data of ", name,": ", nrow(re), " rows"))

return(re)
}
   

In [ ]:
sql3 <- "
WITH med_base as 
(SELECT *
FROM MedicationDispense
WHERE StatusConceptId NOT IN (2989063, 2989065,2989060,2989064)

),

-- EXQC: subject level data
tb4 as
(
   SELECT PersonId, min(DispenseDateTime) as DispenseDataFirstDT, max(DispenseDateTime) as DispenseDataLastDT, count(*) as TotNumDisRec
   FROM med_base
   GROUP BY PersonId
)
SELECT t4.*, (DispenseDataFirstDT -  DispenseDataLastDT) as DISSDAYS
FROM tb4
"

## EX1, EX2 and EXQC

### EX1: per patient, per drug

In [5]:
losartan = codeset_from_prose(con,snapshot, url = "/definitions/doi", variable_name = "losartan")

create_view(losartan, "med_codes")

In [12]:
sql_ex1 <- "

--EX: De-DUP based on essential variables and select subjects with exposure to DOI

--Only medications needed for analysis

--Filter concept maps and names for the medications wanted
WITH map as 
( SELECT mcm.Id, cc.*
FROM  MedicationCodeConceptMap mcm 
JOIN Concept cc on (mcm.CodeConceptId = cc.ConceptId)
WHERE mcm.CodeConceptId IN
(SELECT ConceptId
FROM med_codes
)
),


--Remove error, canceled,declined or stopped dispense 
med_base as 
(SELECT *
FROM MedicationDispense
WHERE StatusConceptId NOT IN (2989063, 2989065,2989060,2989064)

),


-- Join Dispense data with concepts
tb1 as
(
    SELECT DISTINCT d.PersonId, d.DispenseDateTime, --d.Id as DispenseId,
     d.DaysSupply, d.DispenseQuantity, d.DispenseQuantityUOMConceptId,
     mcm.ConceptName as MedicationName, mcm.ConceptDefinition as MedicationDefinition
     FROM med_base d
    INNER JOIN map mcm
      ON mcm.Id = d.CodeConceptMapId
),


---tb2 The overall dispense data needed for final EX data
tb2 as (
    SELECT  d.*, c.ConceptName as DispenseQuantityUOM 
    FROM tb1 d join Concept c on (d.DispenseQuantityUOMConceptId = c.ConceptId)
)

SELECT *
FROM tb2
"

ex1 <- load_sql_table(con,snapshot,query = sql_ex1, output_mode = "sparklyr",view_name = "med_dispense") %>% collect()
display_df(ex1)

In [13]:
nrow(ex1)

In [14]:
filepath = file.path("/public/ADEX-test-chloe/ADEX1.csv")
save_artifacts_data(con, study,ex1, filepath, data_type = "csv")

### EX2: per patient dispense info

In [15]:
sql_ex2 <- "
WITH med_dispense as 
(
    SELECT DISTINCT PersonId, DispenseDateTime
    FROM MedicationDispense
    WHERE DispenseDateTime IS NOT NULL AND
    StatusConceptId NOT IN (2989063, 2989065,2989060,2989064)

),


tb3 as
(
    SELECT PersonId, min(DispenseDateTime) as DSSTDT, max(DispenseDateTime) as DSENDT,
    -- changing distinct dispense Id to distinct dispense date time
     count(distinct DispenseDateTime) as TotNumDisRec
    FROM med_dispense
    GROUP BY PersonId
),

tb4 as (
   SELECT *,DATEDIFF(DSENDT,DSSTDT) as DISSDAYDIFF
   FROM  tb3 
)

SELECT *
FROM tb4
"
ex2 <- load_sql_table(con,snapshot,query = sql_ex2, output_mode = "sparklyr") %>% collect()
display_df(ex2)

In [16]:
nrow(ex2)

In [17]:
filepath = file.path("/public/ADEX-test-chloe/ADEX2.csv")
save_artifacts_data(con, study,ex2, filepath, data_type = "csv")

In [22]:
sql_exqc <- "
--Remove error, canceled,declined or stopped dispense 
WITH med_base as 
(SELECT *
FROM MedicationDispense
WHERE StatusConceptId NOT IN (2989063, 2989065,2989060,2989064)

),

-- EXQC: subject level data
tb4 as
(
   SELECT PersonId, min(DispenseDateTime) as DispenseDataFirstDT, max(DispenseDateTime) as DispenseDataLastDT, count(*) as TotNumDisRec
   FROM med_base
   GROUP BY PersonId
)

SELECT *, DATEDIFF( DispenseDataLastDT,DispenseDataFirstDT) as DISSDAYS
FROM tb4
"

exqc<- load_sql_table(con, snapshot, query = sql_exqc, output_mode = "sparklyr") %>% collect()

display_df(exqc)

In [23]:
nrow(exqc)

In [24]:
filepath = file.path("/public/ADEX-test-chloe/EXQC.csv")
save_artifacts_data(con, study,exqc, filepath, data_type = "csv")

### Encounter Condition

In [6]:
sql <- "
WITH cond as (
SELECT 
    DISTINCT c.*
    FROM Condition c JOIN ConditionCodeConceptMap cm 
    ON c.CodeConceptMapId = cm.Id
    WHERE (cm.SourceConceptId = 2703595 OR cm.SourceConceptId = 2703594)
    AND (OnsetDateTime IS NOT NULL OR RecordedDateTime IS NOT NULL) AND
    cm.CodeConceptId IN 
    (SELECT ConceptId
    FROM
    cond_codes)
    )

SELECT count(*)
FROM cond
"

a <- load_sql_table(con,snapshot, query = sql, view_name = "dementia_cond")